In [16]:
b=2
import pandas as pd
import os
import logging
from collections import Counter

logging.basicConfig(level=logging.DEBUG,format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("hi")


2025-09-02 19:54:35,096 - INFO - hi


In [17]:

import sys
sys.path.append("../metakpick/")

import _utils_tree, _utils_kraken,_classifier, _utils
# from _utils  import get_features_read



In [6]:
workingdir="/vast/blangme2/smajidi5/metagenomics/metakpick_project/MetaKPick/"
in_folder= workingdir+"../files/"
tree_file=in_folder+"nodes.dmp"
tax_genome_file= in_folder + "seqid2taxid.map_tax_uniq"

info, Tree, tax_index, tax_genome, parents, tree_df = _utils_tree.get_tax_info(tree_file,tax_genome_file)    




2025-09-02 19:47:07,248 - DEBUG - Tree file: /vast/blangme2/smajidi5/metagenomics/metakpick_project/MetaKPick/../files/nodes.dmp
2025-09-02 19:47:11,147 - DEBUG - Read the tree file
2025-09-02 19:47:22,925 - DEBUG - Parsed the tree file. Size of info: 2667703, Size of Tree: 241610
2025-09-02 19:47:22,935 - DEBUG - Number of genomes/strains 33646 in kraken index.
2025-09-02 19:47:23,413 - DEBUG - Number of unqiue tax ids:52229, set size after removing duplicates: 353816
2025-09-02 19:47:23,414 - DEBUG - Genome tax not found in kraken tree [3024248, 2778655, 386490]


In [7]:

tax2path, tax2depth = _utils_tree.get_tax2path(tax_genome, info, parents)


2025-09-02 19:47:24,217 - DEBUG - tax not found in kraken tree: [3024248, 2778655, 386490]
2025-09-02 19:47:24,219 - DEBUG - number of paths 33643
2025-09-02 19:47:24,295 - DEBUG - number of tax2path 52229 52229


In [8]:
import pickle
model_folder = "/vast/blangme2/smajidi5/metagenomics/metakpick_project/files/"
model_file = model_folder+"Random_Forest_regression_models20250825_093808.pkl"
loaded_regression_dic= pickle.load(open(model_file, "rb")) # [0] # this is temprorary due to a mistke  remove 

logging.info("Loading the model: "+model_file)
logging.info("Model loaded"+str(loaded_regression_dic))

2025-09-02 19:47:25,337 - INFO - Loading the model: /vast/blangme2/smajidi5/metagenomics/metakpick_project/files/Random_Forest_regression_models20250825_093808.pkl
2025-09-02 19:47:25,341 - INFO - Model loaded{'k17': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k19': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k21': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k23': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k25': RandomForestRegressor(max_features=0.8, max_leaf_nodes=50, n_estimators=1000,
                      n_jobs=20, random_state=14, verbose=True), 'k27': RandomForestRegressor(max_feature

In [9]:

kraken_output_folder="../../../changek/simulatation/classification_pbsim/max15/"
cases_classify =[i.split("_")[0] for i in os.listdir(kraken_output_folder) if i.endswith('_out')]
cases_model= list(loaded_regression_dic.keys())
cases_classify_intersect=sorted(list(set(cases_classify).intersection(set(cases_model))))
kmer_list=[19,21,23,25,27,29, 31]
if kmer_list:   
    kmer_list_set=set(['k'+str(kmer) for kmer in kmer_list])
    cases_classify_intersect=sorted(list(set(cases_classify_intersect).intersection(set(kmer_list_set))))
logging.info("Cases in the input for classification intersected with the kmer list: "+str(cases_classify_intersect))    
    
logging.info("Cases in the input for classification: "+str(cases_classify))
logging.info("Cases in the model: "+str(cases_model))
logging.warning("Working on the intersection of cases in the input for classification and in the model: "+str(cases_classify_intersect))



2025-09-02 19:47:25,350 - INFO - Cases in the input for classification intersected with the kmer list: ['k19', 'k21', 'k23', 'k25', 'k27', 'k29', 'k31']
2025-09-02 19:47:25,351 - INFO - Cases in the input for classification: ['k17', 'k19', 'k21', 'k23', 'k25', 'k27', 'k29', 'k31']
2025-09-02 19:47:25,352 - INFO - Cases in the model: ['k17', 'k19', 'k21', 'k23', 'k25', 'k27', 'k29', 'k31']
2025-09-02 19:47:25,353 - WARNING - Working on the intersection of cases in the input for classification and in the model: ['k19', 'k21', 'k23', 'k25', 'k27', 'k29', 'k31']


In [10]:
logging.info("Reading the kraken kmers for these cases")
read_names_list, kraken_kmers_cases = _utils_kraken.read_kraken_all(cases_classify_intersect, kraken_output_folder)
logging.info("Number of reads in the kraken kmers: "+str(len(kraken_kmers_cases)))
logging.info("Getting the tax depth")
read_tax_depth = _utils_kraken.get_tax_depth(kraken_kmers_cases, info,parents)


2025-09-02 19:47:25,361 - INFO - Reading the kraken kmers for these cases
2025-09-02 19:47:25,362 - INFO - Reading kraken's k-mer count per tax from: ../../../changek/simulatation/classification_pbsim/max15/
2025-09-02 19:47:25,362 - INFO - Reading kraken file for case: k19
2025-09-02 19:50:36,592 - INFO - Number of reads for case: k19 is: 86202
2025-09-02 19:50:36,594 - INFO - Reading kraken file for case: k21
2025-09-02 19:51:33,465 - INFO - Number of reads for case: k21 is: 86202
2025-09-02 19:51:33,466 - INFO - Reading kraken file for case: k23
2025-09-02 19:51:43,367 - INFO - Number of reads for case: k23 is: 86202
2025-09-02 19:51:43,369 - INFO - Reading kraken file for case: k25
2025-09-02 19:51:48,539 - INFO - Number of reads for case: k25 is: 86202
2025-09-02 19:51:48,540 - INFO - Reading kraken file for case: k27
2025-09-02 19:51:52,340 - INFO - Number of reads for case: k27 is: 86202
2025-09-02 19:51:52,341 - INFO - Reading kraken file for case: k29
2025-09-02 19:52:24,795 -

In [75]:
read_names_list2=list(read_names_list)[:1000]
logging.info("Getting the features")
features_cases, feature_names = _utils.get_features_all(read_names_list2, tax2path, kraken_kmers_cases, read_tax_depth, tax2depth, info, parents)
logging.info("Cases in features: "+str(features_cases.keys()))
logging.info("Applying the model")


2025-09-02 21:27:25,275 - INFO - Getting the features
2025-09-02 21:27:25,277 - DEBUG - feature_names are : ['mean_all', 'mean_nonzero', 'max', 'sum', 'mean_exc_tax', 'mean_all/rlen', 'mean_nonzero/rlen', 'max/rlen', 'sum/rlen', 'mean_exc_tax/rlen', 'depth_reported_tax', 'Avg_kmer_consecutive', 'WAvg_kmer_consecutive', 'kmer_reported_tax', 'kmer_tax_above', 'kmer_tax_below', 'kmer_tax/rlen', 'kmer_tax_above/rlen', 'kmer_tax_below/rlen', 'kmer_othertax', 'kmer_othertax_above', 'kmer_othertax_below', 'kmer_othertax/rlen', 'kmer_othertax_above/rlen', 'kmer_othertax_below/rlen', 'diff#kmers_halfRead', 'rlen']
2025-09-02 21:27:25,277 - DEBUG - Extracting features for case: k19
2025-09-02 21:27:25,278 - DEBUG - Extracting features for read idx: 0  out of 1000
2025-09-02 21:27:33,404 - DEBUG - Extracting features for case: k21
2025-09-02 21:27:33,404 - DEBUG - Extracting features for read idx: 0  out of 1000
2025-09-02 21:27:36,172 - DEBUG - Extracting features for case: k23
2025-09-02 21:27:

In [56]:

read_k_prob= _classifier.apply_RF_model(cases_classify_intersect, features_cases,read_names_list2,loaded_regression_dic)
logging.info("Number of reads in the read_k_prob: "+str(len(read_k_prob)))  

logging.info("Getting the best tax")
best_k_dic, estimated_tax_dict = _classifier.get_best_tax(read_k_prob,read_names_list2,kraken_kmers_cases,thr_minprob=0.5) 
logging.info("Writing the estimated tax")

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.


[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.3s finished
2025-09-02 21:03:01,227 - DEBUG - X_input.shape (1000, 27) y_pred.shape (1000,)
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.3s finished
2025-09-02 21:03:01,580 - DEBUG - X_input.shape (1000, 27) y_pred.shape (1000,)
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 ta

In [57]:
truth_file= "../../../changek/simulatation/classification_pbsim/max15/true_tax_max15.csv"
logging.info("Reading the truth file: "+truth_file)
dic_tax_truth = _utils_kraken.read_truth_file(truth_file)
logging.info("Number of reads in the truth file: "+str(len(dic_tax_truth)))


2025-09-02 21:03:03,332 - INFO - Reading the truth file: ../../../changek/simulatation/classification_pbsim/max15/true_tax_max15.csv
2025-09-02 21:03:05,850 - INFO - Number of reads in the truth file: 86202


In [58]:



def calculate_tp_fp(mode,input_dic,dic_tax_truth,info,tree_df,parents,tax_level,tax_index):

    """
    input_dic:
        raw_kraken: kraken_kmers_case     mode:"raw_kraken"
        predicted_tax: predicted_tax_dic  mode:"predicted_tax"
    """
    read_tpfp_dic={"TP":set(),"FP-level-notindex":set(), 'FP-higher-notindex':set(),"FP-level-index":set(), 'FP-higher-index':set(),
               'inconsistent-tool':set(), 'TN':set(),
               'FN':set(),'VP':set(), "truth-level":set(), "no-truth-level":set(),"total_unclassified":set(),'notruth':[]} #  at the level ,'taxNotfoundintool':[]
    
    tid_true_taxlevel_dic={}
    tax_true_all = set(dic_tax_truth.values())
    for tid_true in tax_true_all:
        tid_true_taxlevel_dic[tid_true] = int(_utils_tree.find_tax_level(info,tree_df,parents, tid_true, tax_level))
    tax_predicted_level_dic={}
    tax_predicted_raw=set()    
    for read_name, kraken_read_info in input_dic.items():
        if mode=="raw_kraken":
            tax_predicted_raw.add(kraken_read_info[0])
        elif mode=="predicted_tax":
            tax_predicted_raw.add(kraken_read_info)
    tax_predicted_raw.add(2)
    for tax_predicted in tax_predicted_raw:
        tax_predicted_level_dic[tax_predicted] = int(_utils_tree.find_tax_level(info,tree_df,parents, tax_predicted, tax_level))
    tax_predicted_list_tocheck=[]
    for read_name, kraken_read_info in input_dic.items():
        if read_name not in dic_tax_truth:
            read_tpfp_dic['notruth'].append(read_name)
            continue
        tax_true = dic_tax_truth[read_name]
        if mode=="raw_kraken":
            tax_predicted = kraken_read_info[0]
        elif mode=="predicted_tax":
            tax_predicted = kraken_read_info
        else:
            raise ValueError("mode not supported")

        if tax_predicted==1:
            tax_predicted=2 # todo probably 1 has another meaning in find tax level, that's fine the level should be much deeper 
    
        tax_true_level = tid_true_taxlevel_dic[tax_true]
        tax_predicted_level = tax_predicted_level_dic[tax_predicted]
    
        if tax_predicted_level==-1 and tax_predicted!=0:
            read_tpfp_dic['inconsistent-tool'].add(read_name)
            # tid_tool_taxlevel==-1 we are ignoring these reads if the tax is not in the node.dump 
            # we think these cases are limited and probably happened due to different node.dump in build vs evaluation 
    
        if tax_predicted==0 :
            read_tpfp_dic["total_unclassified"].add(read_name)
        
        if tax_predicted==-1 or tax_true_level < 1:
            tax_predicted_list_tocheck.append(tax_predicted)
        

    
        if tax_true_level ==1 or  tax_true_level==-1 or (tax_predicted_level==-1 and tax_predicted!=0): # true is not at the level, is higher than the level     or true doest not exist in info
            #including 'inconsistent-tool'
            read_tpfp_dic["no-truth-level"].add(read_name)
            
        else:
            read_tpfp_dic["truth-level"].add(read_name)
    
        if tax_true_level >1 and tax_predicted==0:
            if tax_true in tax_index:
                read_tpfp_dic['FN'].add(read_name) # unclassified, among reads with true label at the level 
            else:
                read_tpfp_dic['TN'].add(read_name) # unclassified, among reads with true label at the level 
            
        if tax_true_level>1 and tax_predicted_level==tax_true_level :
            read_tpfp_dic['TP'].add(read_name) # correct
    
        if tax_true_level>1 and tax_predicted>1 and tax_predicted_level==1 :  # tool labled at higher than taxlevel 
            tool_level= tree_df[4][info[tax_predicted]]
            tax_true_toollevel = _utils_tree.find_tax_level(info,tree_df,parents, tax_true, tool_level)
            if tax_true_toollevel == tax_predicted: # at the tool level is correct 
                read_tpfp_dic['VP'].add(read_name) # vague positive, one deeper lvel (ancestror)  match    
            if tax_true_toollevel != tax_predicted:
                if tax_true in tax_index:
                    read_tpfp_dic['FP-higher-index'].add(read_name) # tool labeld at higher level and worng (not a VP) 
                else:
                    read_tpfp_dic['FP-higher-notindex'].add(read_name) # tool labeld at higher level and worng (not a VP) 
    
        if tax_true_level > 1 and tax_predicted_level>1 and  tax_predicted_level != tax_true_level :
            if tax_true in tax_index:
                read_tpfp_dic['FP-level-index'].add(read_name) # tool labeled at the tax level (or lower) and wrong 
            else:
                read_tpfp_dic['FP-level-notindex'].add(read_name) # tool labeled at the tax level (or lower) and wrong             
    if tax_predicted_list_tocheck:
        logging.debug("to check tax_predicted==-1 or tax_true_level < 1: "+str(len(tax_predicted_list_tocheck))+" a few of them are: "+str(tax_predicted_list_tocheck[:20]))        
    return read_tpfp_dic


def calculate_true_k(kraken_kmers_cases,dic_tax_truth,info,tree_df,parents,tax_level,tax_index,read_names_list):
    tp_cases_dic={}
    for case, kraken_kmers in kraken_kmers_cases.items():
        logging.debug('finding true k for case '+case)
        #merged  = _utils_kraken.read_kraken_classification(cases, truth_file, classification_folder )    
        
        
        read_tpfp_dic_case= calculate_tp_fp("raw_kraken",kraken_kmers,dic_tax_truth,info,tree_df,parents,tax_level,tax_index)
        logging.debug("Number of reads in the case TP : "+str(len(read_tpfp_dic_case['TP'])))
        if len(read_tpfp_dic_case['notruth']):
            logging.debug("Number of reads in the case not truth level : "+str(len(read_tpfp_dic_case['notruth']))+" a few examples: "+str(list(read_tpfp_dic_case['notruth'])[:10]))
        tp_cases_dic[case]=read_tpfp_dic_case['TP']

    cases=list(kraken_kmers_cases.keys())
    reads_tp_cases={}
    for read_name in read_names_list:
        reads_tp_cases[read_name]=set()
        for case_k, case in  enumerate(cases): 
            k = int(case[1:3])
            if read_name in tp_cases_dic[case]:
                reads_tp_cases[read_name].add(k)
    logging.debug("Number of reads in the case TP : "+str(len(reads_tp_cases)))
    logging.debug("Number of kmer sizes in the TP for read"+read_name+" : "+str(len(reads_tp_cases[read_name])))
    return reads_tp_cases


In [59]:
reads_tp_cases_all={}
for tax_level in ['species','genus','family','order','class']:
    reads_tp_cases_all[tax_level] = calculate_true_k(kraken_kmers_cases,dic_tax_truth,info,tree_df,parents,tax_level,tax_index,read_names_list)

logging.info("cleaning reported tax for all kmer sizes")


2025-09-02 21:03:05,975 - DEBUG - finding true k for case k19
2025-09-02 21:03:06,887 - DEBUG - to check tax_predicted==-1 or tax_true_level < 1: 8 a few of them are: [548, 196821, 584, 562, 869729, 1134687, 1853130, 196821]
2025-09-02 21:03:06,891 - DEBUG - Number of reads in the case TP : 55825
2025-09-02 21:03:06,892 - DEBUG - finding true k for case k21
2025-09-02 21:03:07,641 - DEBUG - to check tax_predicted==-1 or tax_true_level < 1: 8 a few of them are: [9606, 1547283, 584, 562, 1446707, 573, 240521, 2795216]
2025-09-02 21:03:07,651 - DEBUG - Number of reads in the case TP : 63582
2025-09-02 21:03:07,652 - DEBUG - finding true k for case k23
2025-09-02 21:03:08,324 - DEBUG - to check tax_predicted==-1 or tax_true_level < 1: 8 a few of them are: [2, 186817, 584, 562, 1446707, 573, 2613769, 303]
2025-09-02 21:03:08,333 - DEBUG - Number of reads in the case TP : 63767
2025-09-02 21:03:08,333 - DEBUG - finding true k for case k25
2025-09-02 21:03:08,982 - DEBUG - to check tax_predic

In [60]:


kraken_reportedtax_cases=dict()
kraken_reportedtax_cases['RF']=estimated_tax_dict
for case in cases_classify_intersect:
    kraken_reportedtax_cases[case]={}
    for read_name, kraken_info in kraken_kmers_cases[case].items():
        reported_tax = kraken_info[0]
        kraken_reportedtax_cases[case][read_name]=reported_tax



In [61]:
import random
kraken_reportedtax_cases["Random"]={} 
for read_name in read_names_list2:
    case_random = cases_classify_intersect[random.randint(0, len(cases_classify_intersect)-1)]
    kraken_reportedtax_cases["Random"][read_name]  = kraken_reportedtax_cases[case_random][read_name]      	     

kraken_reportedtax_cases[ "Oracle"]={}
for tax_level in ['species','genus','family','order','class']:
    kraken_reportedtax_cases[ "Oracle"][tax_level]={}
    for read_name in read_names_list:
        true_k_set= reads_tp_cases_all[tax_level][read_name]
        found_true_tax= 0
        if true_k_set:
            case_true_k = 'k'+str(list(true_k_set)[0])
            found_true_tax = kraken_reportedtax_cases[case_true_k][read_name]
        kraken_reportedtax_cases[ "Oracle"][tax_level][read_name]  = found_true_tax           	     


In [62]:
# _utils_kraken

logging.info("Calculating the tp fp")

print("case\tF1\tprecision\trecall\tTP\tFP\tVP\tFN")
for level in ['species']: # ,'genus','family','order','class'
    print("level: "+level)
    for case in list(cases_classify_intersect)+["RF","Random","Oracle"]:
        logging.info("Calculating the tp fp for case: "+case)
        if case=='Oracle':

            dic1_ = {k: v for k, v in kraken_reportedtax_cases[case][level].items() if k in read_names_list2}
            read_tpfp_dic = calculate_tp_fp('predicted_tax',dic1_, dic_tax_truth,info,tree_df,parents,level,tax_index)
        else:
            dic1_ = {k: v for k, v in kraken_reportedtax_cases[case].items() if k in read_names_list2}
            read_tpfp_dic = calculate_tp_fp('predicted_tax',dic1_,dic_tax_truth,info,tree_df,parents,level,tax_index)
    
        logging.info("Number of reads in the TP: "+str(len(read_tpfp_dic['TP'])))

        FP=len(read_tpfp_dic['FP-level-index'])+len(read_tpfp_dic['FP-higher-index'])+len(read_tpfp_dic['FP-level-notindex'])+len(read_tpfp_dic['FP-higher-notindex'])
        recall=len(read_tpfp_dic['TP'])/(len(read_tpfp_dic['TP']) + len(read_tpfp_dic['VP']) + len(read_tpfp_dic['FN']) +FP )
        if len(read_tpfp_dic['TP'])!=0:
            precision=len(read_tpfp_dic['TP'])/(len(read_tpfp_dic['TP']) + FP)
            F1= 2* precision* recall/(precision+recall)
        else:
            precision=0
            F1=0
        print(case,'\t',round(F1,4), round(precision,4),round(recall,4),len(read_tpfp_dic['TP']),FP,len(read_tpfp_dic['VP']),len(read_tpfp_dic['FN']))

        
                

2025-09-02 21:03:50,879 - INFO - Calculating the tp fp
2025-09-02 21:03:50,881 - INFO - Calculating the tp fp for case: k19


case	F1	precision	recall	TP	FP	VP	FN
level: species


2025-09-02 21:03:52,348 - INFO - Number of reads in the TP: 659
2025-09-02 21:03:52,349 - INFO - Calculating the tp fp for case: k21


k19 	 0.6883 0.7202 0.659 659 256 84 1


2025-09-02 21:03:53,781 - INFO - Number of reads in the TP: 755
2025-09-02 21:03:53,782 - INFO - Calculating the tp fp for case: k23


...

RF 	 0.8353 0.9456 0.748 748 43 52 157


2025-09-02 21:04:01,328 - INFO - Calculating the tp fp for case: Oracle


Random 	 0.7821 0.875 0.707 707 101 107 85


2025-09-02 21:04:02,837 - INFO - Number of reads in the TP: 777


Oracle 	 0.8745 1.0 0.777 777 0 0 223


In [41]:
len(dic1_)

0

In [63]:
level='species'
case='RF'
dic1_ = {k: v for k, v in kraken_reportedtax_cases[case].items() if k in read_names_list2}
read_tpfp_dic = calculate_tp_fp('predicted_tax',dic1_,dic_tax_truth,info,tree_df,parents,level,tax_index)
FP=len(read_tpfp_dic['FP-level-index'])+len(read_tpfp_dic['FP-higher-index'])+len(read_tpfp_dic['FP-level-notindex'])+len(read_tpfp_dic['FP-higher-notindex'])
FP

43

In [64]:

level='species'
case='RF'
dic1_ = {k: v for k, v in kraken_reportedtax_cases[case].items() if k in read_names_list2}
read_tpfp_dic = calculate_tp_fp('predicted_tax',dic1_,dic_tax_truth,info,tree_df,parents,level,tax_index)
FP=len(read_tpfp_dic['FP-level-index'])+len(read_tpfp_dic['FP-higher-index'])+len(read_tpfp_dic['FP-level-notindex'])+len(read_tpfp_dic['FP-higher-notindex'])
FP
for i in read_tpfp_dic['FP-level-index']:
    print(i, reads_tp_cases_all['species'][i])
print("---- FP-level-index")
for i in read_tpfp_dic['FP-higher-index']:
    print(i, reads_tp_cases_all['species'][i])
print("---- FP-higher-index ")
for i in read_tpfp_dic['FP-level-notindex']:
    print(i, reads_tp_cases_all['species'][i])
print("---- FP-level-notindex")
for i in read_tpfp_dic['FP-higher-notindex']:
    print(i, reads_tp_cases_all['species'][i])


NZ_CP077387.1__1_1 set()
NZ_LT629768.1__1_1 set()
NZ_CP063844.1__1_2 set()
NZ_CP160302.1__1_2 set()
NZ_CP019626.1__1_1 set()
NZ_CM001889.1__1_2 set()
NC_016639.1__1_2 set()
NZ_CP119130.1__1_1 set()
NZ_CP127083.1__1_1 set()
NZ_AP022021.1__1_1 set()
NZ_CP118677.1__1_1 set()
NZ_PP320298.1__1_1 set()
NZ_CP050009.1__1_2 set()
NZ_CP022377.1__1_2 set()
NZ_NHNB01000108.1__1_1 set()
NZ_MN423361.1__1_1 set()
NZ_CM002882.1__1_1 set()
NZ_NPMK01000019.1__1_1 set()
NZ_CP101701.1__1_1 set()
NC_027622.1__1_1 set()
NZ_CP020338.1__1_1 set()
NZ_PP626394.1__1_1 set()
NZ_CP091379.1__1_1 set()
NC_016621.1__1_1 set()
NZ_MN423362.1__1_1 set()
NZ_AP018734.1__1_1 set()
NZ_CP142091.1__1_1 set()
NC_015469.1__1_1 set()
NZ_CP058628.1__1_1 set()
NZ_CP024905.1__1_2 {25, 23}
NZ_CP004046.1__1_1 set()
NZ_CP009094.1__1_1 set()
NC_042068.1__1_1 set()
NZ_CP104140.1__1_2 {19}
NZ_CP141239.1__1_1 set()
NZ_CP041382.1__1_2 set()
NZ_CP014918.1__1_1 set()
NZ_AOOE01000052.1__1_1 set()
NZ_CP010405.1__1_1 set()
NZ_CP100711.1__1_1 se

In [66]:
r1='NZ_CP083590.1__1_1'#'NZ_PP626394.1__1_1' #list(read_names_list2)[2]
print(r1, dic_tax_truth[r1], kraken_reportedtax_cases['k31'][r1], reads_tp_cases_all['species'][r1])
print(kraken_reportedtax_cases['RF'][r1], kraken_reportedtax_cases['Oracle']['species'][r1])    
[float(round(i,4)) for i in read_k_prob[r1]]

NZ_CP083590.1__1_1 35621 35621 {25, 27, 29, 31}
173858 35621


[0.075, 0.2292, 0.7736, 0.6518, 0.6955, 0.549, 0.4764]

In [67]:
import numpy as np
cases=list(kraken_kmers_cases.keys())
max_val = np.max(read_k_prob[r1])
if max_val >0.5:
    best_k = cases[np.argmax(read_k_prob[r1])]
max_val, best_k

(np.float64(0.7736493212061266), 'k23')

In [83]:
list(f)

[np.float64(1362718.2698412698),
 np.float64(1643.7467881828104),
 np.float64(3239284.0),
 np.float64(85851251.0),
 np.float64(1643.7465967182982),
 np.float64(0.15776363578324307),
 np.float64(0.00019029866653285174),
 np.float64(0.3750160487934259),
 np.float64(9.939109054344312),
 np.float64(0.5592877157939089),
 np.float64(9.0),
 np.float64(4.063694267515924),
 np.float64(29.796178343949045),
 np.float64(10.0),
 np.float64(547.0),
 np.float64(10.0),
 np.float64(0.003402517863218782),
 np.float64(0.18611772711806737),
 np.float64(0.003402517863218782),
 np.float64(48.0),
 np.float64(557.0),
 np.float64(33.0),
 np.float64(0.016332085743450152),
 np.float64(0.18952024498128614),
 np.float64(0.01122830894862198),
 np.float64(10.0),
 np.float64(2939.0)]

In [86]:
for f_idx, feature_name in enumerate(feature_names):
    readid=read_names_list2.index(r1)
    f= features_cases['k31'][readid]
    print(feature_name, round(float(f[f_idx]),4))

mean_all 43036.8
mean_nonzero 4.12
max 173858.0
sum 215184.0
mean_exc_tax 4.12
mean_all/rlen 0.005
mean_nonzero/rlen 0.0
max/rlen 0.0201
sum/rlen 0.0249
mean_exc_tax/rlen 0.0014
depth_reported_tax 9.0
Avg_kmer_consecutive 9.9333
WAvg_kmer_consecutive 76.8
kmer_reported_tax 3.0
kmer_tax_above 293.0
kmer_tax_below 3.0
kmer_tax/rlen 0.001
kmer_tax_above/rlen 0.0997
kmer_tax_below/rlen 0.001
kmer_othertax 5.0
kmer_othertax_above 293.0
kmer_othertax_below 0.0
kmer_othertax/rlen 0.0017
kmer_othertax_above/rlen 0.0997
kmer_othertax_below/rlen 0.0
diff#kmers_halfRead 3.0
rlen 2939.0


In [ ]:
features_cases['k31'][1][-1]

In [ ]:
read_names_list2[:2]



In [ ]:


# tax2path, tax2depth = get_tax2path(tax_genome, info, parents)

# folder_input =workingdir+"../../changek/simulatation/"
# truth_file=folder_input+"true_tax.csv" # generated by generate_training.ipynb

# dic_tax_truth = read_truth_file(truth_file)
# logging.info("Number of reads in the truth file: "+str(len(dic_tax_truth)))

# classification_folder=folder_input+"classification/" 
# cases=[i.split("_")[0] for i in os.listdir(classification_folder) if i.endswith('_out')] #["k"+str(k) for k in range(15,32)]

# kraken_file="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/k31_out"
# dic_kraken=read_kraken_file(kraken_file)
# #print(dic_kraken)

In [ ]:
classification_folder="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/"

cases=[i.split("_")[0] for i in os.listdir(classification_folder) if i.endswith('_out')] #["k"+str(k) for k in range(15,32)]
cases=cases[4:5]+cases[-1:]
cases

In [ ]:

# import pandas as pd
# in_folder= "../../files/"
# tree_file=in_folder+"nodes.dmp"
# tax_genome_file= in_folder + "seqid2taxid.map_tax_uniq"

# info, Tree, tax_index, tax_genome = get_tax_info(tree_file,tax_genome_file)

In [ ]:

# def get_tax_depth(kraken_kmers_cases, info, parents):
#     logging.debug("Getting the tax depth")
#     read_tax_depth={}
#     for case, kraken_kmers in kraken_kmers_cases.items(): 
#         read_tax_depth[case]={}
#         for read_name, kraken_read_info in kraken_kmers.items():
#             reported_tax=kraken_read_info[0]

input_dic= kraken_kmers_cases['k31']
tax_level='species'
read_tpfp_dic= calculate_tp_fp("raw_kraken",input_dic,dic_tax_truth,info,tree_df,parents,tax_level,tax_index)
read_tpfp_dic.keys()


In [ ]:

# def find_true_ksize(cases,merged,info,tree_df,parents,tax_index, readids_max, level='species'):
#     tp_cases_dic={}
#     for case in cases:
#         case1= 'taxon_tool_'+case
#         tp_cases_dic[case]= calculate_tp_fp(merged,info,tree_df,parents,level,case1,tax_index) #+case
#     print(len(tp_cases_dic),len(tp_cases_dic[case]))

#     return tp_cases_dic, true_k

In [ ]:
# a= "C	NZ_OZ243912.1__1_1	1885344	1253	0:557 1885344:2 0:81 1885344:7 0:27 1885344:2 0:28 1885344:3 0:121 1885344:13 0:49 1885344:8 0:35 1885344:6 0:68 1885344:27 0:133 1885344:7 0:49"

# kraken_file="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/k31_out"
# kraken_file_out="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/k31_out_test"
# file_out=open(kraken_file_out,'w')
# file_in=open(kraken_file,'r')
# dic_kraken={}
# for line in file_in: 
#     line_split=line.strip().split("\t")
#     classified ,read_id, tax_krak, read_len, tax_kmers = line_split
#     if read_id in readids_max:
#         file_out.write(line+"\n")
# file_in.close()
# file_out.close()


In [ ]:
# kraken_file="/vast/blangme2/smajidi5/metagenomics/changek/simulatation/classification/max15/k31_out"
# file_o=open(kraken_file,'r')
# dic_kraken={}
# cnt=0
# for line in file_o:
#     cnt+=1
#     line_split=line.strip().split("\t")
#     classified ,read_id, tax_krak, read_len, tax_kmers = line_split

In [ ]:
    # feature_names = ['mean_all', 'mean_nonzero', 'max', 'sum', 'mean_exc_tax', 'mean_all/rlen', 'mean_nonzero/rlen', 'max/rlen', 'sum/rlen', 'mean_exc_tax/rlen', 
    #             'depth_reported_tax', 'Avg_kmer_consecutive', 'WAvg_kmer_consecutive', 'kmer_reported_tax', 'kmer_tax_above', 'kmer_tax_below', 
    #             'kmer_tax/rlen', 'kmer_tax_above/rlen', 'kmer_tax_below/rlen', 'kmer_othertax', 'kmer_othertax_above', 'kmer_othertax_below', 
    #             'kmer_othertax/rlen', 'kmer_othertax_above/rlen', 'kmer_othertax_below/rlen', 'diff#kmers_halfRead']


In [ ]:
# from sklearn import tree
# case='k31'
# individual_tree = loaded_regression_dic[case].estimators_[1]  # Get the first tree (you can choose any index)
# plt.figure(figsize=(30, 30))
# tree.plot_tree(individual_tree, filled=True, rounded=True,fontsize=14, feature_names=feature_names) # , class_names=class_names,
# plt.tight_layout()


# #plt.savefig("/vast/blangme2/smajidi5/metagenomics/_v5__tree_1_4h.png",dpi=300)


In [ ]:
# import matplotlib.pyplot as plt
# for case in ['k19', 'k21', 'k25', 'k31']:
    
#     feature_importances = loaded_regression_dic[case].feature_importances_
#     fig = plt.figure()
    
#     #plt.hist(feature_importances,bins=50)
#     plt.barh(feature_names,feature_importances)
#     #plt.yscale('log')
#     plt.xlabel('Feature importance')
#     #plt.ylabel('No. features')
#     plt.xticks(rotation=85)
#     plt.title(case,fontsize=18)
#     plt.tight_layout()
#     plt.savefig("/vast/blangme2/smajidi5/metagenomics/ـv5d_imporantce"+case+"_.png",dpi=300)

In [ ]:

# def decision_tree_to_code(tree, feature_names):

#     tree_ = tree.tree_
#     feature_name = [
#         feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
#         for i in tree_.feature
#     ]
#     code_txt ="def tree({}):".format(", ".join(feature_names))+"\n"
#     print(code_txt)
#     def recurse_func(node, depth):
#         #code_txt=""
#         indent = "  " * depth
#         if tree_.feature[node] != _tree.TREE_UNDEFINED:
#             name = feature_name[node]
#             threshold = tree_.threshold[node]
#             code_txt = "{}if {} <= {}:".format(indent, name, threshold)+"\n"
#             print(code_txt)
#             recurse_func(tree_.children_left[node], depth + 1)
#             code_txt = "{}else:  # if {} > {}".format(indent, name, threshold)+"\n"
#             print(code_txt)
#             recurse_func(tree_.children_right[node], depth + 1)
#         else:
#             code_txt = "{}return {}".format(indent, tree_.value[node])+"\n"
#             print(code_txt)
#         #print(code_txt)
#         return 1

# from sklearn.tree import _tree
# from sklearn import tree


In [ ]:
# feature_names = ['mean_all', 'mean_nonzero', 'max', 'sum', 'mean_exc_tax', 'mean_all/rlen', 'mean_nonzero/rlen', 'max/rlen', 'sum/rlen', 'mean_exc_tax/rlen', 
#     'depth_reported_tax', 'Avg_kmer_consecutive', 'WAvg_kmer_consecutive', 'kmer_reported_tax', 'kmer_tax_above', 'kmer_tax_below', 
#     'kmer_tax/rlen', 'kmer_tax_above/rlen', 'kmer_tax_below/rlen', 'kmer_othertax', 'kmer_othertax_above', 'kmer_othertax_below', 
#     'kmer_othertax/rlen', 'kmer_othertax_above/rlen', 'kmer_othertax_below/rlen', 'diff#kmers_halfRead']


In [ ]:

# import matplotlib.pyplot as plt

# tree_idx=1 
# regr=loaded_regression_dic['k19']    
# # for case,regr in regr_dic.items():
# #     for tree_idx in range(num_trees):
# individual_tree = regr.estimators_[tree_idx]  # Get the first tree (you can choose any index)
# decision_tree_to_code(individual_tree,feature_names)
# plt.figure(figsize=(30, 30))
# tree.plot_tree(individual_tree, filled=True ,rounded=True,fontsize=14, feature_names=feature_names) # , class_names=class_names,
# #plt.savefig(model_folder+"/tree_"+case+"_"+str(tree_idx)+".png",dpi=200)
